# Accessing FAO WaPOR data

In this exercise we look into downloading Precipitation, Evapotranspiration and biomass production data for a region from the global WaPOR database.

More information on WaPOR: https://www.fao.org/in-action/remote-sensing-for-water-productivity/project/en

WaPOR data portal: https://data.apps.fao.org/wapor/?lang=en

Before we start, we need to install some required python packages.

In [ ]:
!pip install ipykernel ipyleaflet geopandas geojson --quiet

The following cell ensures a correct download of the data from Google Cloud Storage.

In [ ]:
import os
os.environ['GS_NO_SIGN_REQUEST'] = 'YES'

In [ ]:
# Available data in WaPOR v3:

import pandas as pd
from vito_agri_tutorials.fao.wapor import WAPOR3_SUPPORTED_VARIABLES

df_vars = pd.DataFrame.from_dict(WAPOR3_SUPPORTED_VARIABLES)
df_vars = df_vars.transpose()
df_vars

In [ ]:
# Get your polygon of interest based on the previous exercise

# NOTE: To be able to request PCP data, your polygon needs to cover at least one 5x5 km grid cell.

from pathlib import Path
import geopandas as gpd

in_dir = Path('./results/asis')
infile = in_dir / 'polygon_wapor.gpkg'
gdf = gpd.read_file(infile)
gdf

## OR use the following code to draw a polygon on the map
# from vito_agri_tutorials.utils.map import ui_map
# map = ui_map(geometry_type='polygon')
## Execute this in a next cell
# gdf = map.get_objects()
# gdf
 

In [ ]:
# Get precipitation data for the polygon

#NOTE: PCP data comes in its original resolution of 5 km x 5 km grid cells.

from vito_agri_tutorials.fao.wapor import extract_wapor_data, compute_area_stats, plot_stats

out_dir = Path('./results/wapor')

variable = "L1-PCP-D"
start_date = '2022-01-01'
end_date = '2024-12-31'
write_tif = True

pcp = extract_wapor_data(variable,
                         gdf,
    out_dir,
    start_date,
    end_date=end_date,
    write_tif=write_tif,
    debug=True,)

pcp_stats = compute_area_stats(pcp)
pcp_mean = plot_stats(pcp_stats, variable, "mean")


In [ ]:
# Get AETI data for the polygon

from vito_agri_tutorials.fao.wapor import extract_wapor_data, compute_area_stats, plot_stats

out_dir = Path('./results/wapor')

variable = "L1-AETI-D"
start_date = '2022-01-01'
end_date = '2024-12-31'
write_tif = True

aeti = extract_wapor_data(variable,
                         gdf,
    out_dir,
    start_date,
    end_date=end_date,
    write_tif=write_tif,
    debug=True,)

aeti_stats = compute_area_stats(aeti)
aeti_mean = plot_stats(aeti_stats, variable, "mean")

In [ ]:
# Plot PCP and AETI data together

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, figsize=(10, 5))
pcp_mean.plot(ax=ax, label='PCP', color='blue', kind='line')
aeti_mean.plot(ax=ax, label='AETI', color='red', kind='line')
plt.title('PCP and AETI data')
ax.set_xlabel('Date')
ax.set_ylabel('mm/day')
ax.legend(labels=['AETI', 'PCP'])
plt.show()

# Now also try plotting the NPP timeseries in green on the same plot...

In [ ]:
# Get total biomass production data for the polygon

from vito_agri_tutorials.fao.wapor import extract_wapor_data, compute_area_stats, plot_stats

out_dir = Path('./results/wapor')

variable = "L1-TBP-A"
start_date = '2022-01-01'
end_date = '2024-12-31'
write_tif = True

tbp = extract_wapor_data(variable,
                         gdf,
    out_dir,
    start_date,
    end_date=end_date,
    write_tif=write_tif,
    debug=True,)

tbp_stats = compute_area_stats(tbp)
tbp_mean = plot_stats(tbp_stats, variable, "mean", plot=False)

In [ ]:
from matplotlib import pyplot as plt

figs, ax = plt.subplots(1, figsize=(12, 6))

tbp_mean['test'].plot.bar(ax=ax, label='TBP', color='green')
plt.title('Total Biomass Production')
ax.set_xlabel('Date')
ax.set_ylabel('kg/ha')
